In [38]:
import alphaVantageAPI as AV
from tqdm.autonotebook import tqdm
import sqlite3
import pandas as pd

In [39]:
from marketml import project_root
with sqlite3.connect("../data/db/stocks.sqlite") as conn:
    df = pd.read_sql("SELECT DISTINCT ticker FROM exchanges", conn, columns=["ticker"])
    tickers = df["ticker"].values

[I 201202 22:02:29 __init__:15] Project directory: D:\Projects\MarketML


In [40]:
API_KEY="CW8VRSTFQJ70G8ZS"

av = AV.AlphaVantage(
    api_key=API_KEY,
    output_size="full",
    # export=False,
    # export_path="../data/interim",
    clean=True
)

def get_ticker_table(stock):
    table_path = project_root.joinpath(f"data/interim/{stock}.csv")
    if not table_path.exists():
        with sqlite3.connect("../data/db/stocks.sqlite") as conn:
            table = pd.read_sql(f'SELECT * FROM prices WHERE ticker=="{stock}"', conn, columns=["ticker"])
        if len(table) == 0:
            table = av.data(symbol=stock, function="DA")
        table.to_csv(str(table_path))



# with ThreadPoolExecutor() as threads:
#     extractions = threads.map(get_ticker_table, tickers)
for ticker in tqdm(tickers[::-1], total=len(tickers)):
    get_ticker_table(ticker)

ValueError: DataFrame constructor not properly called!